Import Panda und SQLalchemy

In [8]:
import pandas as pd
import sqlalchemy as sql

csv-File importieren.

In [9]:
sh_2022 = pd.read_csv("data/SH_2022.csv", delimiter= ";", encoding= 'windows-1252')
sh_2022.head(20)
sh_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6004 entries, 0 to 6003
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   EP               6004 non-null   int64  
 1   Kapitel          6004 non-null   int64  
 2   MGTG             2968 non-null   float64
 3   Titel            6004 non-null   int64  
 4   Zweckbestimmung  6004 non-null   object 
 5   Ist 2020 T€      6004 non-null   object 
 6   Soll 2021 T€     6004 non-null   object 
 7   Ansatz 2022 T€   6004 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 375.4+ KB


Drop die ersten zwei Columns, die nicht gebraucht werden

In [10]:
sh_2022 = sh_2022.drop(['Ist 2020 T€','Soll 2021 T€'], axis=1)

Columns names anpassen, Sonderzeichen entfernen, in tausend Euro entfernen

In [11]:
sh_2022.columns = sh_2022.columns.str.replace(" ", "_")
sh_2022.columns = sh_2022.columns.str.lower()
sh_2022.rename(columns={"ansatz_2022_t€":"ansatz_2022_euro"}, inplace=True)
sh_2022


,ep,kapitel,mgtg,titel,zweckbestimmung,ansatz_2022_euro
0,1,101,NaN,11902,Kostenbeiträge für private Benutzung verwaltun...,"0,0"
1,1,101,NaN,11904,Erlöse aus dem Verkauf unbrauchbarer oder entb...,"0,0"
2,1,101,NaN,11999,Vermischte Einnahmen,"0,0"
3,1,101,NaN,12401,"Einnahmen aus Vermietung, Verpachtung und Nutzung","0,0"
4,1,101,NaN,12402,Einnahmen aus der Inanspruchnahme von Leistung...,"18,1"
...,...,...,...,...,...,...
5999,16,1614,2.0,81222,Erwerb von Hard- und Software,"1.500,0"
6000,16,1614,5.0,81225,Erwerb von Hard- und Software,"0,0"
6001,16,1614,6.0,81226,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...","0,0"
6002,16,1614,7.0,81227,Erwerb digitaler Technologien,"1.200,0"


Values ins passende Format setzten und leading-0 einfügen, wo sie automatisch entfernt wurde

In [12]:
sh_2022['ep'] = sh_2022['ep'].astype(str)
sh_2022['kapitel'] = sh_2022['kapitel'].astype(str)
sh_2022['titel'] = sh_2022['titel'].astype(str)
sh_2022['titel'] = sh_2022['titel'].apply(lambda x:x.zfill(5))
sh_2022['ep'] = sh_2022['ep'].apply(lambda x:x.zfill(2))
sh_2022['kapitel'] = sh_2022['kapitel'].str[-2:]
sh_2022

,ep,kapitel,mgtg,titel,zweckbestimmung,ansatz_2022_euro
0,01,01,NaN,11902,Kostenbeiträge für private Benutzung verwaltun...,"0,0"
1,01,01,NaN,11904,Erlöse aus dem Verkauf unbrauchbarer oder entb...,"0,0"
2,01,01,NaN,11999,Vermischte Einnahmen,"0,0"
3,01,01,NaN,12401,"Einnahmen aus Vermietung, Verpachtung und Nutzung","0,0"
4,01,01,NaN,12402,Einnahmen aus der Inanspruchnahme von Leistung...,"18,1"
...,...,...,...,...,...,...
5999,16,14,2.0,81222,Erwerb von Hard- und Software,"1.500,0"
6000,16,14,5.0,81225,Erwerb von Hard- und Software,"0,0"
6001,16,14,6.0,81226,"Erwerb von Geräten, Ausstattungs- und Ausrüstu...","0,0"
6002,16,14,7.0,81227,Erwerb digitaler Technologien,"1.200,0"


Bei den Zahlen . und , Systematik angepasst und *1000 genommen und von string zu float und dann INT umgewandelt.

In [13]:
sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].str.replace('.', '').str.replace(",", ".").astype(float)
sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].apply(lambda x:int(x*1000))

/var/folders/mh/m321ff8j0k50t4vlfqk08n540000gn/T/ipykernel_23428/798112430.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sh_2022.ansatz_2022_euro = sh_2022['ansatz_2022_euro'].str.replace('.', '').str.replace(",", ".").astype(float)


Titel in gruppe und counter gesplittet, gruppe und counter eingefügt

In [14]:
sh_2022["gruppe"] = sh_2022["titel"].str[:3]
sh_2022["counter"] = sh_2022["titel"].str[3:]
sh_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6004 entries, 0 to 6003
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ep                6004 non-null   object 
 1   kapitel           6004 non-null   object 
 2   mgtg              2968 non-null   float64
 3   titel             6004 non-null   object 
 4   zweckbestimmung   6004 non-null   object 
 5   ansatz_2022_euro  6004 non-null   int64  
 6   gruppe            6004 non-null   object 
 7   counter           6004 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 375.4+ KB


jahr und state_id eingefügt

In [15]:
sh_2022.insert(8, "jahr", 2022)
sh_2022.insert(9, "state_id", 15)
sh_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6004 entries, 0 to 6003
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ep                6004 non-null   object 
 1   kapitel           6004 non-null   object 
 2   mgtg              2968 non-null   float64
 3   titel             6004 non-null   object 
 4   zweckbestimmung   6004 non-null   object 
 5   ansatz_2022_euro  6004 non-null   int64  
 6   gruppe            6004 non-null   object 
 7   counter           6004 non-null   object 
 8   jahr              6004 non-null   int64  
 9   state_id          6004 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 469.2+ KB


titel gedropt, da redundant

In [16]:
sh_2022 = sh_2022.drop("titel", axis=1)
sh_2022.head(20)

,ep,kapitel,mgtg,zweckbestimmung,ansatz_2022_euro,gruppe,counter,jahr,state_id
0,01,01,NaN,Kostenbeiträge für private Benutzung verwaltun...,0,119,02,2022,15
1,01,01,NaN,Erlöse aus dem Verkauf unbrauchbarer oder entb...,0,119,04,2022,15
2,01,01,NaN,Vermischte Einnahmen,0,119,99,2022,15
3,01,01,NaN,"Einnahmen aus Vermietung, Verpachtung und Nutzung",0,124,01,2022,15
4,01,01,NaN,Einnahmen aus der Inanspruchnahme von Leistung...,18100,124,02,2022,15
5,01,01,NaN,Einnahmen aus Veranstaltungen nach dem Nutzung...,8500,124,03,2022,15
6,01,01,NaN,Einnahmen aus der Beteiligung Dritter bei Kuns...,0,129,01,2022,15
7,01,01,1.0,Erstattung von Personalkosten,0,281,02,2022,15
8,01,01,NaN,Entnahme aus der Rücklage für Sachausgaben,0,359,01,2022,15
9,01,01,NaN,Entnahme aus der Rücklage für personalwirtscha...,0,359,02,2022,15


anhand der gruppe einnahmen und ausgaben in den Zahlen definiert

In [17]:
sh_2022['ansatz_2022_euro'] =sh_2022[['ansatz_2022_euro', 'gruppe']].apply(lambda x:-x[0] if int(x[1]) > 400 else x[0], axis=1)


mgtg gedropt, hat keinen mehrwert und viele nan

In [18]:
sh_2022 = sh_2022.drop(['mgtg'], axis=1)


einmal angucken, da schön

In [19]:
sh_2022.head()

,ep,kapitel,zweckbestimmung,ansatz_2022_euro,gruppe,counter,jahr,state_id
0,01,01,Kostenbeiträge für private Benutzung verwaltun...,0,119,02,2022,15
1,01,01,Erlöse aus dem Verkauf unbrauchbarer oder entb...,0,119,04,2022,15
2,01,01,Vermischte Einnahmen,0,119,99,2022,15
3,01,01,"Einnahmen aus Vermietung, Verpachtung und Nutzung",0,124,01,2022,15
4,01,01,Einnahmen aus der Inanspruchnahme von Leistung...,18100,124,02,2022,15
